In [1]:
!pip install -U git+https://github.com/faustomorales/keras-ocr.git
!pip install imgaug
!pip install -U opencv-python
%tensorflow_version 2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/faustomorales/keras-ocr.git to /tmp/pip-req-build-5zg0jk8x
  Running command git clone -q https://github.com/faustomorales/keras-ocr.git /tmp/pip-req-build-5zg0jk8x
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 930 kB 4.2 MB/s 
     |████████████████████████████████| 603 kB 48.7 MB/s 
     |████████████████████████████████| 9.5 MB 46.5 MB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.9.0.post2+40bb9bd-py3-none-any.whl size=42441 sha256=395ed38f34cb967c37e2cf7ce5f322a5231d7070ae68addf74cc2ca4a5851340
  Stored in directory: /tmp/pip-ephem-wheel-cache-lu8z5_to/wheels/20/d0/06/ed290995144ec8dde71d3a91d0d590145445c01be60ea70c9b
  Created wheel for validators: filename=validators-0.19

In [2]:
import random
import string
import math
import itertools
import os

import numpy as np

import imgaug
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection

import keras_ocr

train_labels = keras_ocr.datasets.get_icdar_2013_recognizer_dataset(
    cache_dir='.'
)

Looking for ./icdar2013/Challenge2_Training_Task12_Images.zip
Looking for ./icdar2013/Challenge2_Training_Task2_GT.zip


In [3]:
train_labels = [(filepath, box, word.lower()) for filepath, box, word in train_labels]

In [4]:
recognizer = keras_ocr.recognition.Recognizer()
recognizer.compile()

Looking for /root/.keras-ocr/crnn_kurapan.h5


In [5]:
batch_size = 8
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.GammaContrast(gamma=(0.25, 3.0)),
])

train_labels, validation_labels = sklearn.model_selection.train_test_split(train_labels, train_size=0.8, random_state=42)

(training_image_gen, training_steps), (validation_image_gen, validation_steps) = [
    (
        keras_ocr.datasets.get_recognizer_image_generator(
            labels=labels,
            height=recognizer.model.input_shape[1],
            width=recognizer.model.input_shape[2],
            alphabet=recognizer.alphabet,
            augmenter=augmenter
        ),
        len(labels) // batch_size
    ) for labels, augmenter in [(train_labels, augmenter), (validation_labels, None)]     
]
training_gen, validation_gen = [
    recognizer.get_batch_generator(
        image_generator=image_generator,
        batch_size=batch_size
    )
    for image_generator in [training_image_gen, validation_image_gen]
]

In [6]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False),
    tf.keras.callbacks.ModelCheckpoint('recognizer_icdar2013.h5', monitor='val_loss', save_best_only=True),
]
recognizer.training_model.fit(
    x=training_gen,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    validation_data=validation_gen,
    callbacks=callbacks,
    epochs=50,
    verbose=1
)

68 / 679 instances have illegal characters.
Epoch 1/50
84/84 [==============================] - 71s 572ms/step - loss: 4.1102 - val_loss: 5.1102
Epoch 2/50
84/84 [==============================] - 45s 535ms/step - loss: 2.1943 - val_loss: 5.1035
Epoch 3/50
84/84 [==============================] - 46s 550ms/step - loss: 1.8755 - val_loss: 3.6451
Epoch 4/50
84/84 [==============================] - 46s 552ms/step - loss: 1.3503 - val_loss: 4.2010
Epoch 5/50
84/84 [==============================] - 45s 538ms/step - loss: 1.3533 - val_loss: 2.0285
Epoch 6/50
84/84 [==============================] - 46s 548ms/step - loss: 1.1759 - val_loss: 3.0119
Epoch 7/50
84/84 [==============================] - 42s 508ms/step - loss: 1.0980 - val_loss: 4.1106
Epoch 8/50
84/84 [==============================] - 46s 554ms/step - loss: 0.7638 - val_loss: 3.0062
Epoch 9/50
84/84 [==============================] - 46s 547ms/step - loss: 0.6104 - val_loss: 2.6696
Epoch 10/50
84/84 [============================

In [ ]:
from google.colab import files
files.download('/content/recognizer_icdar2013.h5') 